In [1]:
from scipy.stats import multivariate_normal
import numpy as np
def pdf(points, mean, cov, prior):
    points, mean, cov = np.asarray(points), np.asarray(mean), np.asarray(cov)
    prior = np.asarray(prior)
    n, d = points.shape
    k, d_1 = mean.shape
    k_2, d_2, d_3 = cov.shape
    k_3, = prior.shape
    assert d == d_1 == d_2 == d_3
    assert k == k_2 == k_3, "%s %s %s should be equal" % (k, k_2, k_3)

    # Compute probabilities
    prob = []
    for i in range(k):
        if prior[i] < 1 / k ** 3:
            prob.append(np.zeros(n))
        else:
            prob.append(
                prior[i] *
                multivariate_normal.pdf(
                    mean=mean[i], cov=cov[i], x=points))
    prob = np.transpose(prob)  # n x k
    # Normalize cluster probabilities of each point
    prob = prob / np.sum(prob, axis=1, keepdims=True)  # n x k

    assert prob.shape == (n, k)
    assert np.allclose(prob.sum(axis=1), 1)
    return prob

In [2]:
def most_likely(points, mean, cov, prior):
    prob = pdf(points, mean, cov, prior)
    return np.argmax(prob, axis=1)

In [41]:
def em(points, k, epsilon, mean=None):
    points = np.asarray(points)
    n, d = points.shape
    # Initialize and validate mean
    if mean is None:
        # Randomly pick k points
        mean = points[np.random.choice(range(n), size=k, replace=False)]
        print("Boom")
        print(mean.shape)
    # Validate input
    mean = np.asarray(mean)
    k_, d_ = mean.shape
    assert k == k_
    assert d == d_
    # Initialize cov, prior
    cov = np.asarray([np.identity(d)]*k)
    prior = np.ones(shape=(k))/k
    tired = False
    old_mean = np.zeros_like(mean)
    while not tired:
        old_mean[:] = mean

        # Expectation step
        exp = pdf(points, mean, cov, prior)
        print(exp.shape)
        # Maximization step
        prior = exp.sum(axis=0)/n
        mean =  (np.dot(points.T, exp)/exp.sum(axis=0)).T
        print("boom2")
        print(mean.shape)
        num = 0
        for i in range(k):
            for j in range(n):
                sub = points[j, :]-mean[i, :]
                num += exp[j, i] * sub * sub.T
        prior = (num/exp.sum(axis=0)).T
        print(prior.shape)
        # Finish condition
        dist = np.sqrt(((mean - old_mean) ** 2).sum(axis=1))
        tired = True #np.all(dist < epsilon)

    # Validate output
    assert mean.shape == (k, d)
    assert cov.shape == (k, d, d)
    assert prior.shape == (k,)
    return mean, cov, prior


em(np.random.rand(150, 4), 3, 0.05)

Boom
(3, 4)
(150, 3)
boom2
(3, 4)


ValueError: operands could not be broadcast together with shapes (4,) (3,) 